# Cleaning data


In [ ]:
import pandas as pd
import numpy as np

import datetime as dt

import warnings
warnings.filterwarnings('ignore')

**!! The current csv file doent contain whats expected to run the notebook**

In the typical DS workflow we: 
1. Access the data
2. Explore and process data
3. Extract Insights
4. Report Insights

Dirty data can appear due to wrong encoding, duplicate entries, wrong processing, etc... If the raw data is garbage, we cannot expect but garbage to be output.

## Ensuring the right data types

First thing to check is that our variables have the right datatype. 

### Strings to integers

In [ ]:
ride_sharing = pd.read_csv('../data/ride_sharing_new.csv')

print(ride_sharing.info())

print(ride_sharing['user_type'].describe())

ride_sharing['user_type_cat'] = ride_sharing['user_type'].astype('category')

assert ride_sharing['user_type_cat'].dtype == 'category'

ride_sharing['user_type_cat'].describe()

In [ ]:
ride_sharing['duration']

In [ ]:
ride_sharing['duration_trim'] = ride_sharing['duration'].str.strip('minutes')

ride_sharing['duration_time'] = ride_sharing['duration_trim'].astype('int')

assert ride_sharing['duration_time'].dtype == 'int'

print(ride_sharing[['duration','duration_trim','duration_time']])
print(ride_sharing.duration_time.mean())

## Data range constraints

When the range of a certain variable is known, it can happen that there are observations outside the range.

What to do with out of range data:
- Dropping data: go for this iff the amount of impacted data is low and after understanding the root cause of the issues.
- Setting custom minimums or maximums
- Setting custom value depending on business assumptions



In [ ]:
ride_sharing.describe()

In [ ]:
# The loaded dataframe dont contain tiresizes, but the idea is to cap the values going beyond the expected range to its max value.
ride_sharing['tire_sizes'] = ride_sharing['tire_sizes'].astype('int')
ride_sharing.loc[ride_sharing['tire_sizes'] > 27, 'tire_sizes'] = 27
ride_sharing['tire_sizes'] = ride_sharing['tire_sizes'].astype('category')


## Handling duplicates

Duplicates happen. Often they are not exact duplicates.
They are originated because of: 
- Human errors
- Bugs and design errors on business processes or pipelines
- Joining/merging data sources

How to find duplicates?
.duplicated()

How to remove duplicates?
.drop_duplicates() 

We can use the duplicates to be removed to fulfill or improve the remaining records by using .groupby() and agg()



In [ ]:
duplicates = ride_sharing.duplicated(subset=['ride_id'], keep=False)

duplicated_rides = ride_sharing[duplicates].sort_values('ride_id')

print(duplicated_rides[['ride_id','duration','user_birth_year']])

In [ ]:
ride_dup = ride_sharing.drop_duplicates()

statistics = {'user_birth_year': 'min', 'duration': 'mean'}

ride_unique = ride_dup.groupby('ride_id').agg(statistics).reset_index()

duplicates = ride_unique.duplicated(subset = 'ride_id', keep = False)
duplicated_rides = ride_unique[duplicates == True]

assert duplicated_rides.shape[0] == 0£

## Categories and membership constraints

Categorical data represent variables whose values belong to predefined set of categories. Variables like "Marriage Status", "Household Income Category" or "Loan Status" belong to this category.

When feeding models, these variables are often converted into a numeric representation.

Categorical variables cannot have values outside of the predefined ones.

Inconsistencies in categorical variables may happen due to a variety of reasons: data entry errors, parsing errors...

### Identifying inconsistent categories

Before fixing the inconsistent categories we have to identify them:

In [ ]:
categories = ['A', 'B', 'C']
observations = ['A', 'B', 'D'] 

inconsistent_categories = set(observations).difference(categories)

print(inconsistent_categories)

We can now use the 'inconsistent_categories' with the isin pandas method to find observations with inconsistent values.

How do we treat these problems?

### Dropping inconsistent categories

In [ ]:
categories = pd.DataFrame({'cleanliness': {
                                0: 'Clean',
                                1: 'Average',
                                2: 'Somewhat clean',
                                3: 'Somewhat dirty',
                                4: 'Dirty'
                            },'safety': {
                                0: 'Neutral',
                                1: 'Very safe',
                                2: 'Somewhat safe',
                                3: 'Very unsafe',
                                4: 'Somewhat unsafe'
                            },'satisfaction': {
                                0: 'Very satisfied',
                                1: 'Neutral',
                                2: 'Somewhat satisfied',
                                3: 'Somewhat unsatisfied',
                                4: 'Very unsatisfied'
                            }
                          }
)

airlines = pd.read_csv('../data/airlines_final.csv')

print('Cleanliness: ', airlines['cleanliness'].unique(), "\n")
print('Safety: ', airlines['safety'].unique(), "\n")
print('Satisfaction: ', airlines['satisfaction'].unique(), "\n")

In [ ]:
cat_clean = set(airlines['cleanliness']).difference(categories['cleanliness'])
cat_clean_rows = airlines['cleanliness'].isin(cat_clean)
airlines[~cat_clean_rows]

### Other errors when dealing with categorical variables

#### Value inconsistency
Due to trailing spaces, wrong casing, spaces...

We can str.upper() or str.lower() case to solve casing issues.

str.strip() will solve the trailing spaces problem

In [ ]:
print(airlines['dest_region'].unique())
print(airlines['dest_size'].unique())

In [ ]:
airlines['dest_region'] = airlines['dest_region'].str.lower()
airlines['dest_region'] = airlines['dest_region'].replace({'eur':'europe'})

airlines['dest_size'] = airlines['dest_size'].str.strip()

In [ ]:
print(airlines['dest_region'].unique())
print(airlines['dest_size'].unique())

#### Too many categories

Often collapsable into few.

For splitting into new categories there are two options: qcut (dumber) and cut (smarter). 

If we want to collapse already existing categories grouping them we can use the replace method.

In [ ]:
airlines.day.unique()

In [ ]:
airlines.wait_min.describe()

In [ ]:
label_ranges = [0, 60, 180, np.inf]
label_names = ['short', 'medium', 'long']

airlines['wait_type'] = pd.cut(airlines['wait_min'], bins = label_ranges, 
                                labels = label_names)

In [ ]:
mappings = {'Monday':'weekday', 'Tuesday':'weekday', 'Wednesday': 'weekday', 
            'Thursday': 'weekday', 'Friday': 'weekday', 
            'Saturday': 'weekend', 'Sunday': 'weekend'}

airlines['day_week'] = airlines['day'].replace(mappings)

## Cleaning text data

Text data is very frequent.

Typical errors are typos, casing, excesive length, different formats...

Regular expressions is a powerful tool for fixing these issues.

In [ ]:
airlines['full_name'] = airlines['full_name'].str.replace("Dr.","")
airlines['full_name'] = airlines['full_name'].str.replace("Mr.","")
airlines['full_name'] = airlines['full_name'].str.replace("Miss","")
airlines['full_name'] = airlines['full_name'].str.replace("Ms.","")

assert airlines['full_name'].str.contains('Ms.|Mr.|Miss|Dr.').any() == False

In [ ]:
resp_length = airlines['survey_response'].str.len()

airlines_survey = airlines[resp_length > 40]

assert airlines_survey['survey_response'].str.len().min() > 40

print(airlines_survey['survey_response'])

## Uniformity

Ensuring unit uniformity is paramount. Currency, temperatures, distances... are prone to have different units through datasets.

Dates can have multiple formats too. Sometimes, specific dates can be ambiguous.

Its very important to understand where the data is coming from.


In [ ]:
banking = pd.read_csv('../data/banking_dirty.csv')

In [ ]:
acct_eu = banking['acct_cur'] == 'euro'

# Convert acct_amount where it is in euro to dollars
banking.loc[acct_eu, 'acct_amount'] = banking.loc[acct_eu, 'acct_amount'] * 1.1

# Unify acct_cur column by changing 'euro' values to 'dollar'
banking.loc[acct_eu, 'acct_cur'] = 'dollar'

# Assert that only dollar currency remains
assert banking['acct_cur'].unique() == 'dollar'

In [ ]:
print(banking['account_opened'].head())

banking['account_opened'] = pd.to_datetime(banking['account_opened'],
                                           # Infer datetime format
                                           infer_datetime_format = True,
                                           # Return missing value for error
                                           errors = 'coerce') 

banking['acct_year'] = banking['account_opened'].dt.strftime('%Y')

print(banking['acct_year'])

### Cross Field Validation 

Cross field validation refers to the usage of multiple fields in a dataset to sanity check data integrity. This is often required when merging different datasets comming from different sources.

In case of inconsistencies, many options can be taken into account, all of them requiring a deep understanding about where the data is comming from.


In [ ]:
fund_columns = ['fund_A', 'fund_B', 'fund_C', 'fund_D']

inv_equ = banking[fund_columns].sum(axis=1) == banking['inv_amount']

consistent_inv = banking[inv_equ]
inconsistent_inv = banking[~inv_equ]

print("Number of inconsistent investments: ", inconsistent_inv.shape[0])

In [ ]:
today = dt.date.today()
banking['birth_date'] = pd.to_datetime(banking['birth_date'])

ages_manual = 2020 - banking['birth_date'].dt.year

# Find rows where age column == ages_manual
age_equ = banking['Age'] == ages_manual

# Store consistent and inconsistent data
consistent_ages = banking[age_equ]
inconsistent_ages = banking[~age_equ]

# Store consistent and inconsistent data
print("Number of inconsistent ages: ", inconsistent_ages.shape[0])

In [ ]:
banking

### Completeness and missing data

Missing data occurs when no data value is stored for a variable in an observation. Can be represented as NaN, NA, 0...

There are multiple reasons behind missing data.


In [ ]:
# To get an idea about missingness
banking.isna().sum()

There is no null value in this dataframe

The **missingno** package allows us to better understand our missing data.

In [ ]:
import missingno as msno
import matplotlib.pyplot as plt

msno.matrix(banking)
plt.show()

Missing values can happen:
- Completely at random: completely due to randomness
- At random: there is a relationship between missing data and other observed values
- Not at random: there is a systematic relationship between missing data and unobserved values

Again, there are many ways to proceed with missing data: dropping it, imputing means, medians or modes, imputing based on machine learning models, impute them by hand if we have sufficient domain knowledge...

## Record Linkage

**Minimum edit distance**: least possible amount of steps needed to transition from one string to another, operations being insertion, deletion, substitution or transposition. The smaller the minimum edit distance, the closer the 2 words are. There are several algorithms to calculate this distance.

Several packages to calculate these distances: thefuzz, nltk, recordlinkage...


In [ ]:
restaurants = pd.read_csv('../data/restaurants_L2_dirty.csv')
restaurants

In [ ]:
from thefuzz import process

unique_types = restaurants.type.unique()

print(process.extract('asian', unique_types, limit = len(unique_types)))
print(process.extract('american', unique_types, limit = len(unique_types)))
print(process.extract('italian', unique_types, limit = len(unique_types)))

In [ ]:
print(restaurants['type'].unique())

In [ ]:
matches = process.extract('italian', restaurants['type'], limit=len(restaurants))

# Inspect the first 5 matches
print(matches[0:5])

In [ ]:
matches = process.extract('italian', restaurants['type'], limit=len(restaurants.type))

for match in matches:
  if match[1] >= 80:
    restaurants.loc[restaurants['type'] == match[0]] = 'italian'

In [ ]:
for cuisine in categories:  
  # Create a list of matches, comparing cuisine with the cuisine_type column
  matches = process.extract(cuisine, restaurants['type'], limit=len(restaurants.type))

  # Iterate through the list of matches
  for match in matches:
     # Check whether the similarity score is greater than or equal to 80
    if match[1] >= 80:
      # If it is, select all rows where the cuisine_type is spelled this way, and set them to the correct cuisine
      restaurants.loc[restaurants['type'] == match[0]] = cuisine
      
# Inspect the final result
print(restaurants['type'].unique())

**Record Linkage** is the act of linking data from different sources regarding the same entity. Generally, we clean two or more DataFrames, generate pairs of potentially matching records, score these pairs according to string similarity and other similarity metrics, and link them. All of these steps can be achieved with the recordlinkage package

https://recordlinkage.readthedocs.io/en/latest/

### Generating pairs
First step for record linkage: composing candidate pairs for further comparison.
Blocking is the technique to reduce the amount of pairs based on variables having the same value on both observations of the same pair.


In [ ]:
import recordlinkage 

indexer = recordlinkage.Index()

# Block pairing on cuisine_type
indexer.block('type')

# Generate pairs
pairs = indexer.index(restaurants, restaurants_new)

### Comparing fields

In [ ]:
comp_cl = recordlinkage.Compare()

# Find exact matches on city, cuisine_types 
comp_cl.exact('city', 'city', label='city')
comp_cl.exact('type', 'type', label = 'cuisine_type')

# Find similar matches of rest_name
comp_cl.string('rest_name', 'rest_name', label='name', threshold = 0.8) 

potential_matches = comp_cl.compute(pairs, restaurants, restaurants_new)
print(potential_matches)


### Linking dataframes

In [ ]:
matches = potential_matches[potential_matches.sum(axis = 1) >= 3]

# Get values of second column index of matches
matching_indices = matches.index.get_level_values(1)

# Subset restaurants_new based on non-duplicate values
non_dup = restaurants_new[~restaurants_new.index.isin(matching_indices)]

# Append non_dup to restaurants
full_restaurants = restaurants.append(non_dup)
print(full_restaurants)

